In [1]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
loader = UnstructuredPDFLoader("Art Brain.pdf")

In [3]:
data = loader.load()
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


You have 1 document(s) in your data
There are 71830 characters in your document


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

print (f'Now you have {len(texts)} documents')

Now you have 76 documents


In [5]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

C:\Users\trung\anaconda3\envs\pdfreader\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
OPENAI_API_KEY = 'your openai api key'
PINECONE_API_KEY = 'your pinecone api key'
PINECONE_API_ENV = 'your pincone api env'

In [7]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [8]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchaintest"

In [9]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [10]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [11]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [26]:
query = "is an artwork a type of storytelling"
docs = docsearch.similarity_search(query, include_metadata=True)

In [27]:
chain.run(input_documents=docs, question=query)

' Yes, an artwork can be a type of storytelling. Art allows the audience to experience a story not only through hearing it but can also be seeing it or even feeling it. Through visuals, music, and other elements, an artwork can evoke more story based on the unique interpretation of each audience.'